In [2]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from datetime import datetime

# Create a SQLite database
conn = sqlite3.connect('financial_app.db')
cursor = conn.cursor()

# Create a transactions table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        description TEXT,
        amount REAL
    )
''')
conn.commit()

class FinancialApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Financial Tracking App")

        # Variables
        self.date_var = tk.StringVar()
        self.description_var = tk.StringVar()
        self.amount_var = tk.StringVar()

        # Entry Fields
        tk.Label(root, text="Date:").grid(row=0, column=0)
        ttk.Entry(root, textvariable=self.date_var).grid(row=0, column=1)

        tk.Label(root, text="Description:").grid(row=1, column=0)
        ttk.Entry(root, textvariable=self.description_var).grid(row=1, column=1)

        tk.Label(root, text="Amount:").grid(row=2, column=0)
        ttk.Entry(root, textvariable=self.amount_var).grid(row=2, column=1)

        # Buttons
        ttk.Button(root, text="Add Transaction", command=self.add_transaction).grid(row=3, column=0, columnspan=2, pady=10)

        # Treeview to display transactions
        self.tree = ttk.Treeview(root, columns=("Date", "Description", "Amount"), show="headings")
        self.tree.heading("Date", text="Date")
        self.tree.heading("Description", text="Description")
        self.tree.heading("Amount", text="Amount")
        self.tree.grid(row=4, column=0, columnspan=2)

        # Load existing transactions
        self.load_transactions()

    def add_transaction(self):
        # Get values from entry fields
        date = self.date_var.get()
        description = self.description_var.get()
        amount = self.amount_var.get()

        # Validate input
        if not description or not amount:
            tk.messagebox.showerror("Error", "All fields are required!")
            return

        try:
            # Convert amount to float
            amount = float(amount)
        except ValueError:
            tk.messagebox.showerror("Error", "Invalid amount!")
            return

        # Insert into the database
        cursor.execute("INSERT INTO transactions (date, description, amount) VALUES (?, ?, ?)", (date, description, amount))
        conn.commit()

        # Clear entry fields
        self.date_var.set("")
        self.description_var.set("")
        self.amount_var.set("")

        # Reload transactions
        self.load_transactions()

    def load_transactions(self):
        # Clear existing data in the treeview
        for item in self.tree.get_children():
            self.tree.delete(item)

        # Fetch transactions from the database
        cursor.execute("SELECT date, description, amount FROM transactions")
        transactions = cursor.fetchall()

        # Insert transactions into the treeview
        for transaction in transactions:
            self.tree.insert("", "end", values=transaction)

if __name__ == "__main__":
    root = tk.Tk()
    app = FinancialApp(root)
    root.mainloop()

# Close the database connection when the app is closed
conn.close()
